# Checking soilcores files

In [18]:
import numpy as np
import os
import pandas as pd

In [33]:
df = pd.read_excel("data_info/MNHT_paths_v1.xlsx")

In [34]:
# remove the rows where the root_path is nan and reset the index
df = df.loc[~df["root_path"].isna()].reset_index(drop=True)
df

,label ID,root_path,filename
0,N1002,MU\SOIL64_8_10_20,SOIL_FRIT_8_10_20Series_003_Bone 0.5 SCAN 1.2
1,N1003,MU\SOIL64_4_8_5_20_3,SOIL_FRIT_8_5_20_3Series_003_Bone 0.5 SCAN 7
2,N1006,MU\SOIL64_2_8_5_20_1,SOIL_FRIT_8_5_20_1Series_004_Bone 0.5 SCAN 4
3,N1008,MU\SOIL64_3_8_5_20_2,SOIL_FRIT_8_5_20_2Series_003_Bone 0.5 SCAN 5
4,N1009,MU\SOIL64_8_5_20,SOIL_FRIT_8_5_20Series_004_Bone 0.5 SCAN 2
...,...,...,...
170,N3310,MU\DICOM_07\0000\002,NaN
171,N3311,MU\SOIL64_8_19_20,SOIL_FRIT_8_19_20Series_002_Bone 0.5 SCAN 1
172,N3316,MU\SOIL_8_10_20_2,SOIL_8_10_20_2Series_004_Bone 0.5 SCAN 6
173,N3317,MU\SOIL64_2_8_7_20_1,SOIL_FRIT_8_7_20_1Series_004_Bone 0.5 SCAN 4


In [35]:
ini_root_path = ".."

all_found = True

# df_row = df.iloc[0]
for index, df_row in df.iterrows():
    labelID = df_row["label ID"]
    root_path = df_row["root_path"]
    file_name = df_row["filename"]

    if not pd.isna(file_name):
        dicom_path = os.path.join(ini_root_path, root_path, file_name + ".dcm")
        labels_path = os.path.join(ini_root_path, root_path, "labels", file_name + "_corelabels" + ".nii.gz")
        descriptions_path = os.path.join(ini_root_path, root_path, "labels", file_name + "_descriptions" + ".txt")
    else:
        dicom_path = None
        labels_dir = os.path.join(ini_root_path, root_path, "labels")
        prefix = os.path.basename(root_path) + "_"  # e.g. 002_corelabels.nii.gz
        labels_path = os.path.join(labels_dir, prefix + "corelabels.nii.gz")
        descriptions_path = os.path.join(labels_dir, prefix + "descriptions.txt")

    # Load descriptions (ITK-SNAP format: IDX  R G B A  VIS MSH  "LABEL")
    label_descriptions = {}
    if os.path.isfile(descriptions_path):
        with open(descriptions_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                parts = line.split()
                if not parts:
                    continue
                try:
                    idx = int(parts[0])
                    # Label is the last quoted string
                    start = line.find('"')
                    end = line.rfind('"')
                    if start != -1 and end != -1 and end > start:
                        label_descriptions[idx] = line[start + 1 : end]
                    else:
                        label_descriptions[idx] = parts[-1]
                except (ValueError, IndexError):
                    pass

    if not labelID in label_descriptions.values():
        print(f"Label ID {labelID} not found in descriptions")
        all_found = False

if all_found:
    print("All labels found in descriptions")

All labels found in descriptions


True

In [15]:
# check if the value is nan
pd.isna(df["root_path"].loc[0])

True